In [ ]:
!pip install -U transformers datasets scipy scikit-learn ipywidgets seqeval

In [1]:
import random
from IPython.display import display, HTML

import numpy as np
import pandas as pd
import datasets
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer

In [2]:
task = "ner"
model_checkpoint = "klue/roberta-base"
batch_size = 16

In [4]:
datasets = load_dataset("klue", task)

Reusing dataset klue (/root/.cache/huggingface/datasets/klue/ner/1.0.0/55ff8f92b7a4b9842be6514ce0b4b5295b46d5e493f8bb5760da4be717018f90)


In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['ner_tags', 'sentence', 'tokens'],
        num_rows: 21008
    })
    validation: Dataset({
        features: ['ner_tags', 'sentence', 'tokens'],
        num_rows: 5000
    })
})

In [6]:
datasets["train"][0]

{'ner_tags': [12,
  12,
  12,
  2,
  3,
  3,
  3,
  3,
  3,
  12,
  2,
  3,
  12,
  12,
  12,
  12,
  2,
  3,
  3,
  3,
  3,
  12,
  12,
  12,
  2,
  3,
  3,
  3,
  3,
  12,
  12,
  12,
  8,
  9,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12,
  12],
 'sentence': '특히 <영동고속도로:LC> <강릉:LC> 방향 <문막휴게소:LC>에서 <만종분기점:LC>까지 <5㎞:QT> 구간에는 승용차 전용 임시 갓길차로제를 운영하기로 했다.',
 'tokens': ['특',
  '히',
  ' ',
  '영',
  '동',
  '고',
  '속',
  '도',
  '로',
  ' ',
  '강',
  '릉',
  ' ',
  '방',
  '향',
  ' ',
  '문',
  '막',
  '휴',
  '게',
  '소',
  '에',
  '서',
  ' ',
  '만',
  '종',
  '분',
  '기',
  '점',
  '까',
  '지',
  ' ',
  '5',
  '㎞',
  ' ',
  '구',
  '간',
  '에',
  '는',
  ' ',
  '승',
  '용',
  '차',
  ' ',
  '전',
  '용',
  ' ',
  '임',
  '시',
  ' ',
  '갓',
  '길',
  '차',
  '로',
  '제',
  '를',
  ' ',
  '운',
  '영',
  '하',
  '기',
  '로',
  ' ',
  '했',
  '다',
  '.']}

In [7]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(num_classes=13, names=['B-DT', 'I-DT', 'B-LC', 'I-LC', 'B-OG', 'I-OG', 'B-PS', 'I-PS', 'B-QT', 'I-QT', 'B-TI', 'I-TI', 'O'], names_file=None, id=None), length=-1, id=None)

In [8]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['B-DT',
 'I-DT',
 'B-LC',
 'I-LC',
 'B-OG',
 'I-OG',
 'B-PS',
 'I-PS',
 'B-QT',
 'I-QT',
 'B-TI',
 'I-TI',
 'O']

In [9]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(datasets["train"])

,ner_tags,sentence,tokens
0,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-LC, I-LC, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",뭐 하나 빠지지 않는 드라마ㅋㅋ 러브라인도 <한국:LC>처럼 대놓고 스토리고 뭐고 주인공들 엮기에 급급하지 않고 아슬아슬(?)하게 해서 더 좋았다,"[뭐, , 하, 나, , 빠, 지, 지, , 않, 는, , 드, 라, 마, ㅋ, ㅋ, , 러, 브, 라, 인, 도, , 한, 국, 처, 럼, , 대, 놓, 고, , 스, 토, 리, 고, , 뭐, 고, , 주, 인, 공, 들, , 엮, 기, 에, , 급, 급, 하, 지, , 않, 고, , 아, 슬, 아, 슬, (, ?, ), 하, 게, , 해, 서, , 더, , 좋, 았, 다]"
1,"[O, O, O, B-PS, I-PS, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PS, I-PS, I-PS, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","평소 <코난:PS> 자주 보는데 완전 재미있었고, <유명한:PS>이 대박이었다!!!!!!!!","[평, 소, , 코, 난, , 자, 주, , 보, 는, 데, , 완, 전, , 재, 미, 있, 었, 고, ,, , 유, 명, 한, 이, , 대, 박, 이, 었, 다, !, !, !, !, !, !, !, !]"
2,"[O, O, O, O, O, O, O, O, O, B-QT, I-QT, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-TI, I-TI, I-TI, O, O, O, O, O, O, O, O, O, O, O, B-QT, I-QT, O, O, O, O, O, O, O, O, B-QT, I-QT, O, O, O, O, O, O]",그냥 고전적으로 <2D:QT> 로 쭉 가는게 맞는거 같고 무슨 애니매이션 영화가 <30분:TI>이냐 ㅋㅋ어이없어서 <0점:QT> 주고 싶은데 <1점:QT>주고 간다.,"[그, 냥, , 고, 전, 적, 으, 로, , 2, D, , 로, , 쭉, , 가, 는, 게, , 맞, 는, 거, , 같, 고, , 무, 슨, , 애, 니, 매, 이, 션, , 영, 화, 가, , 3, 0, 분, 이, 냐, , ㅋ, ㅋ, 어, 이, 없, 어, 서, , 0, 점, , 주, 고, , 싶, 은, 데, , 1, 점, 주, 고, , 간, 다, .]"
3,"[O, O, O, B-OG, I-OG, I-OG, O, O, B-OG, I-OG, I-OG, I-OG, I-OG, I-OG, I-OG, I-OG, I-OG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-QT, I-QT, O, O, O, O, O, O, B-QT, I-QT, I-QT, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-QT, I-QT, O, O, O, O, O, O, B-QT, I-QT, O, O, O, O, O, O, O, O, B-QT, I-QT, I-QT, O, O, O, O, ...]","한편 <복지부:OG>는 <에볼라 긴급구호대:OG>에 소속될 민간 의료인력으로 예비인력을 포함해 의사 <8명:QT>, 간호사 <12명:QT>을 선발했으며, 이 가운데 의사 <6명:QT>, 간호사 <9명:QT>이 군 의료진 <15명:QT>과 함께 <세 차례:QT>로 나뉘어 <시에라리온:LC>에 파견될 예정이다.","[한, 편, , 복, 지, 부, 는, , 에, 볼, 라, , 긴, 급, 구, 호, 대, 에, , 소, 속, 될, , 민, 간, , 의, 료, 인, 력, 으, 로, , 예, 비, 인, 력, 을, , 포, 함, 해, , 의, 사, , 8, 명, ,, , 간, 호, 사, , 1, 2, 명, 을, , 선, 발, 했, 으, 며, ,, , 이, , 가, 운, 데, , 의, 사, , 6, 명, ,, , 간, 호, 사, , 9, 명, 이, , 군, , 의, 료, 진, , 1, 5, 명, 과, , 함, 께, ...]"
4,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-LC, I-LC, I-LC, I-LC, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",k팝써바이벌은 세계인들이 주목한 행사로써 <대한민국:LC> 위상이 올라가는 계기가 됐다,"[k, 팝, 써, 바, 이, 벌, 은, , 세, 계, 인, 들, 이, , 주, 목, 한, , 행, 사, 로, 써, , 대, 한, 민, 국, , 위, 상, 이, , 올, 라, 가, 는, , 계, 기, 가, , 됐, 다]"
5,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-QT, I-QT, O, O, O, O, O, O, O, O]",이런 영화를 위한 장르명을 새로 <하나:QT> 지어줘야한다.,"[이, 런, , 영, 화, 를, , 위, 한, , 장, 르, 명, 을, , 새, 로, , 하, 나, , 지, 어, 줘, 야, 한, 다, .]"
6,"[B-PS, I-PS, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",<보영:PS>은 아직 신인때라 그런지 조금 미성숙한 연기임..그리고 영화 내용이나 연출도 많이 떨어지내요...,"[보, 영, 은, , 아, 직, , 신, 인, 때, 라, , 그, 런, 지, , 조, 금, , 미, 성, 숙, 한, , 연, 기, 임, ., ., 그, 리, 고, , 영, 화, , 내, 용, 이, 나, , 연, 출, 도, , 많, 이, , 떨, 어, 지, 내, 요, ., ., .]"
7,"[O, O, O, O, O, O, O, O, O, B-PS, I-PS, O, B-PS, I-PS, O, O, O, O, O, O, B-PS, I-PS, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-DT, I-DT, O, O, O, O, O, O, O, O, O, O, O]",전설적인 록그룹 <유투:PS>(<U2:PS>)의 보컬 <보노:PS>가 자전거 사고 후유증으로 다시는 기타를 연주할 수 없을지도 모른다고 <1일:DT>(현지시간) 밝혔다.,"[전, 설, 적, 인, , 록, 그, 룹, , 유, 투, (, U, 2, ), 의, , 보, 컬, , 보, 노, 가, , 자, 전, 거, , 사, 고, , 후, 유, 증, 으, 로, , 다, 시, 는, , 기, 타, 를, , 연, 주, 할, , 수, , 없, 을, 지, 도, , 모, 른, 다, 고, , 1, 일, (, 현, 지, 시, 간, ), , 밝, 혔, 다, .]"
8,"[B-DT, I-DT, I-DT, I-DT, I-DT, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]",<올해 9월:DT>개봉작 뭔러너? 예고보는데 이 영화 생각나길래..,"[올, 해, , 9, 월, 개, 봉, 작, , 뭔, 러, 너, ?, , 예, 고, 보, 는, 데, , 이, , 영, 화, , 생, 각, 나, 길, 래, ., .]"
9,"[O

In [11]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [12]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [2, 13472, 10211, 2036, 16, 25419, 11376, 23828, 17219, 30062, 9963, 5, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [2, 13472, 10211, 2036, 16, 25419, 11376, 23828, 17219, 30062, 9963, 12405, 2010, 4412, 20641, 2036, 90, 13135, 2041, 18, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
example = datasets["train"][4]
print(example["tokens"])

['팀', ' ', '조', '연', '들', '의', ' ', '탁', '월', '한', ' ', '연', '기', '와', ' ', '홍', '금', '보', '의', ' ', '녹', '슬', '지', '않', '은', ' ', '관', '록', '의', ' ', '무', '술', '!']


In [15]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', '팀', '조', '연', '들', '의', '탁', '월', '한', '연', '기', '와', '홍', '금', '보', '의', '녹', '슬', '지', '않', '은', '관', '록', '의', '무', '술', '!', '[SEP]']


In [16]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(33, 28)

In [17]:
print(tokenized_input.word_ids())

[None, 0, 2, 3, 4, 5, 7, 8, 9, 11, 12, 13, 15, 16, 17, 18, 20, 21, 22, 23, 24, 26, 27, 28, 30, 31, 32, None]


In [18]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

28 28


In [19]:
label_all_tokens = True

In [20]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [21]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[2, 1813, 1969, 1437, 856, 594, 1283, 848, 991, 553, 1026, 1129, 1904, 1091, 1037, 1956, 578, 1282, 1421, 1258, 1038, 1558, 1175, 645, 1540, 653, 1583, 25, 207, 615, 545, 1421, 793, 1324, 1468, 1632, 1537, 1468, 1510, 1325, 551, 647, 1632, 991, 1545, 1022, 1471, 1437, 1889, 645, 991, 1902, 809, 18, 3], [2, 1891, 617, 842, 1258, 1885, 1023, 1498, 743, 1088, 727, 1187, 1891, 1518, 1873, 1511, 801, 809, 18, 3], [2, 1889, 1583, 1038, 1504, 1437, 1933, 1421, 793, 548, 1969, 1156, 25, 558, 1022, 1564, 594, 1335, 809, 3], [2, 1504, 1141, 1038, 723, 1498, 1793, 823, 991, 1238, 1268, 1093, 1235, 570, 1263, 1173, 1091, 823, 1890, 1250, 645, 1517, 812, 23, 645, 793, 1388, 1495, 991, 25, 558, 1479, 545, 1936, 1397, 1498, 1857, 1689, 578, 860, 809, 18, 3], [2, 1823, 1552, 1431, 882, 1503, 1762, 1479, 1891, 1431, 645, 1453, 1932, 641, 1160, 1503, 767, 1319, 1583, 1380, 1497, 606, 992, 1503, 1088, 1299, 5, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [22]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

In [23]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this mode

In [24]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [25]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [29]:
metric = load_metric("seqeval")

In [30]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'PS': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [31]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [32]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [33]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results